<a href="https://colab.research.google.com/github/nicolemichaud03/Recipe-Recommender-System/blob/main/Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone Project - Recipe/Meal Recommendation System

In [1]:
!pip install scikit-surprise
# !conda install -y -c conda-forge scikit-surprise # If you use conda on a non-Colab environment



In [2]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from surprise.model_selection import train_test_split
import io
from surprise import SVD, BaselineOnly, SVDpp, NMF
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the user and item data and viewing it:

In [4]:
user_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_interactions.csv')

In [5]:
recipe_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_recipes.csv')

In [6]:
user_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [7]:
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [9]:
#dropping columns any additional columns to prepare df for Suprise (can only take in user_id, item_id, and ratings)
recipe_df = recipe_data.drop(columns=['contributor_id', 'submitted', 'tags', 'nutrition', 'description', 'ingredients', 'n_steps', 'steps', 'n_ingredients'])
recipe_df.head()

,name,id,minutes
0,arriba baked winter squash mexican style,137739,55
1,a bit different breakfast pizza,31490,30
2,all in the kitchen chili,112140,130
3,alouette potatoes,59389,45
4,amish tomato ketchup for canning,44061,190


In [10]:
# read in values as Surprise dataset
from surprise import Reader, Dataset

reader = Reader()
recipe_df = Dataset.load_from_df(recipe_df, reader)


In [11]:
print(recipe_df)

In [ ]:
user_df = user_data.drop(columns=['date', 'review'])
user_df.head()

In [ ]:
user_df = Dataset.load_from_df(user_df, reader)
user_df

In [ ]:
#additional preprocessed dataframes from original source, don't know if I will use these
#recipes_PP = pd.read_csv('data/PP_recipes.csv')
#users_PP = pd.read_csv('data/PP_users.csv')

In [ ]:
#pickled_map = pd.read_pickle('data/ingr_map.pkl')
#pickled_map

## Data Exploration

In [ ]:
user_data.info()

In [ ]:
print(len(user_data['user_id'].unique()))

There are 226,570 unique users in this dataset

In [ ]:
recipe_data.info()

In [ ]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps'])

In [ ]:
print(len(recipe_data['id'].unique()))

There are 231,637 unique recipes in this dataset.

With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

## Preprocessing

The text data in description needs to be cleaned to ensure all punctuation is removed and words are all lowercase. The text data in the other columns looks pretty well cleaned already, but we still need to remove stopwords.

In [22]:
# Creating a function to perform cleaning steps at once
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    #text = no_nums.sub('', text)
    text = no_bad_chars.sub(' ', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
tags_cleaned = recipe_data['tags'].apply(clean_text)
tags_cleaned

0         '60 minutes less' 'time make' 'course' 'main i...
1         '30 minutes less' 'time make' 'course' 'main i...
2         'time make' 'course' 'preparation' 'main dish'...
3         '60 minutes less' 'time make' 'course' 'main i...
4         'weeknight' 'time make' 'course' 'main ingredi...
                                ...                        
231632    'ham' '60 minutes less' 'time make' 'course' '...
231633    '15 minutes less' 'time make' 'course' 'prepar...
231634    '60 minutes less' 'time make' 'course' 'main i...
231635    '30 minutes less' 'time make' 'course' 'prepar...
231636    '30 minutes less' 'time make' 'course' 'prepar...
Name: tags, Length: 231637, dtype: object

In [24]:
GF = []
for row in recipe_data['tags']:
    if "gluten-free" in row : GF.append(1)
    elif "gluten free" in row : GF.append(1)
    else: GF.append(0)

In [25]:
recipe_data['GF'] = GF

In [26]:
recipe_data['GF'].value_counts()

0    225894
1      5743
Name: GF, dtype: int64

In [27]:
#Ingredient lists for diet filtering:
vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']

In [28]:
recipe_data['vegetarian'] = None
recipe_data['vegan'] = None

In [29]:
vege_pattern = '|'.join(vegetarian)
vegan_pattern = '|'.join(vegan)


recipe_data.vegetarian = recipe_data.ingredients.str.contains(vege_pattern)
recipe_data.vegan = recipe_data.ingredients.str.contains(vegan_pattern)

In [30]:
recipe_data.head()

,name,id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,False,True
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,True,True
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,True,True
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,False,True
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,False,False


In [31]:
new_vals = {'False': 1, 'True': 0}

In [32]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   name           231636 non-null  object
 1   id             231637 non-null  int64 
 2   minutes        231637 non-null  int64 
 3   tags           231637 non-null  object
 4   n_steps        231637 non-null  int64 
 5   description    226658 non-null  object
 6   ingredients    231637 non-null  object
 7   n_ingredients  231637 non-null  int64 
 8   GF             231637 non-null  int64 
 9   vegetarian     231637 non-null  bool  
 10  vegan          231637 non-null  bool  
dtypes: bool(2), int64(5), object(4)
memory usage: 16.3+ MB


In [33]:
recipe_data['vegetarian'] = recipe_data['vegetarian'].astype(str)
recipe_data['vegetarian'] = recipe_data['vegetarian'].replace({'False': 1, 'True': 0})

In [34]:
recipe_data['vegan'] = recipe_data['vegan'].astype(str)
recipe_data['vegan'] = recipe_data['vegan'].replace({'False': 1, 'True': 0})

In [35]:
recipe_data.head()

,name,id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,1,0
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,0,0
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,0,0
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,1,0
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,1,1


In [36]:
recipe_data['vegetarian'].value_counts()

1    134596
0     97041
Name: vegetarian, dtype: int64

In [37]:
recipe_data['vegan'].value_counts()

0    197961
1     33676
Name: vegan, dtype: int64

In [38]:
#Making column names match and merging dfs to classify user diets based on the recipes they've used
recipe_data = recipe_data.rename(columns = {'id': 'recipe_id'})

user_diets = pd.merge(user_data, recipe_data, on='recipe_id', how='left')



In [39]:
user_diets = user_diets.drop(columns=['date', 'review', 'name', 'minutes', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients'])
user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan
0,38094,40893,4,0,0,0
1,1293707,40893,5,0,0,0
2,8937,44394,4,0,1,0
3,126440,85009,5,0,0,0
4,57222,85009,5,0,0,0


In [40]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132367 entries, 0 to 1132366
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   user_id     1132367 non-null  int64
 1   recipe_id   1132367 non-null  int64
 2   rating      1132367 non-null  int64
 3   GF          1132367 non-null  int64
 4   vegetarian  1132367 non-null  int64
 5   vegan       1132367 non-null  int64
dtypes: int64(6)
memory usage: 60.5 MB


In [41]:
len(user_diets['user_id'].unique())

226570

In [42]:
#user_diets = (user_diets.groupby(['user_id']).agg({'recipe_id': lambda x: x.tolist(), 'rating': lambda x: x.tolist(), 'GF': sum, 'vegetarian':sum, 'vegan': sum}).reset_index())
#user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",11,57,8
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",20,630,114
2,1581,[341050],[5],0,0,0
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",3,35,9
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",3,4,2


In [42]:
user_diets['is_vegetarian'] = None
user_diets['is_vegan'] = None
user_diets['is_GF'] = None
user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF
0,38094,40893,4,0,0,0,None,None,None
1,1293707,40893,5,0,0,0,None,None,None
2,8937,44394,4,0,1,0,None,None,None
3,126440,85009,5,0,0,0,None,None,None
4,57222,85009,5,0,0,0,None,None,None


In [48]:
user_diets['recipe_totals'] = user_diets[user_diets['recipe_id']].sum()

KeyError: "None of [Int64Index([ 40893,  40893,  44394,  85009,  85009, 120345, 120345, 120345,\n            134728, 134728,\n            ...\n             82303,  82303,  82303,  82303,  54493,  72730, 386618,  78003,\n             78003,  78003],\n           dtype='int64', length=1132367)] are in the [columns]"

In [44]:
user_diets.head()

,user_id,recipe_id,rating,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF
0,38094,40893,4,0,0,0,None,None,None
1,1293707,40893,5,0,0,0,None,None,None
2,8937,44394,4,0,1,0,None,None,None
3,126440,85009,5,0,0,0,None,None,None
4,57222,85009,5,0,0,0,None,None,None


In [45]:
user_diets['is_vegetarian'] = np.where(user_diets['vegetarian'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)

user_diets.head()

KeyError: 'recipe_totals'

In [47]:
user_diets['is_vegetarian'].value_counts()

1    114857
0    111713
Name: is_vegetarian, dtype: int64

In [46]:
user_diets['is_vegan'] = np.where(user_diets['vegan'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)
user_diets['is_vegan'].value_counts()

0    206565
1     20005
Name: is_vegan, dtype: int64

In [47]:
user_diets['is_GF'] = np.where(user_diets['GF'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)
user_diets['is_GF'].value_counts()

0    220978
1      5592
Name: is_GF, dtype: int64

In [50]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226570 entries, 0 to 226569
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        226570 non-null  int64 
 1   recipe_id      226570 non-null  object
 2   rating         226570 non-null  object
 3   GF             226570 non-null  int64 
 4   vegetarian     226570 non-null  int64 
 5   vegan          226570 non-null  int64 
 6   is_vegetarian  226570 non-null  int64 
 7   is_vegan       226570 non-null  int64 
 8   is_GF          226570 non-null  int64 
 9   recipe_totals  226570 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 17.3+ MB


In [51]:
user_diets_only = user_diets[['user_id', 'recipe_id', 'rating', 'is_GF', 'is_vegetarian', 'is_vegan']]
recipe_diets = user_diets[[ 'recipe_id', 'rating', 'GF', 'vegetarian', 'vegan']]

In [52]:
user_diets_only.head()

,user_id,recipe_id,rating,is_GF,is_vegetarian,is_vegan
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...","[5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,0,0
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",0,1,0
2,1581,[341050],[5],0,0,0
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...","[4, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, ...",0,0,0
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...","[5, 4, 5, 5, 5, 5, 4, 5, 3, 3, 5, 5, 1, 5, 5, ...",0,0,0


In [57]:
#attempting to make separate user and item df's that also include the extra diet features
user_diets_only = user_diets_only.explode(['user_id','recipe_id', 'rating'])
user_diets_only.head()

ValueError: columns must have matching element counts

In [51]:
user_diets_full = user_diets.explode('recipe_id')

In [52]:
user_features = user_diets_full.drop(columns=['recipe_id', 'rating', 'GF', 'vegetarian', 'vegan', 'recipe_totals'])
user_features.head()

,user_id,is_vegetarian,is_vegan,is_GF
0,1533,0,0,0
0,1533,0,0,0
0,1533,0,0,0
0,1533,0,0,0
0,1533,0,0,0


In [53]:
#Create new dataframes for users in each diet type (and for those who fall into none of the diet types)
#vege_users = user_diets[['user_id', 'recipe_id', 'is_vegetarian']].loc[user_diets['is_vegetarian'] == 1]
#vege_users.head()

In [54]:
# vegan_users = user_diets[['user_id', 'recipe_id', 'is_vegan']].loc[user_diets['is_vegan'] == 1]
# vegan_users.info()

In [55]:
# gf_users = user_diets[['user_id', 'recipe_id', 'is_GF']].loc[user_diets['is_GF'] == 1]
# gf_users.info()


In [56]:
# no_diet_users = user_diets[['user_id', 'recipe_id', 'recipe_totals']].loc[(user_diets['is_vegan'] == 0) & (user_diets['is_vegetarian'] == 0) & (user_diets['is_GF'] == 0)]
# no_diet_users.head()

## Modeling

Seeing what model works best for this data:

In [57]:
# #Iterate over all potential algorithms to see which one performs best

# benchmark = []


# algorithms = [SVD(), SVDpp(), NMF(), BaselineOnly()]

# print ("Attempting: ", str(algorithms), '\n\n\n')

# for algorithm in algorithms:
#     #print("Starting: " ,str(algorithm))
#     # Perform cross validation
#     results = cross_validate(algorithm, user_df, measures=['RMSE'], cv=3, verbose=False)
#     # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)

#     # Get results & append algorithm name
#     tmp = pd.DataFrame.from_dict(results).mean(axis=0)
#     tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
#     benchmark.append(tmp)
#     print("Done: " ,str(algorithm), "\n\n")

# print ('\n\tDONE\n')

In [58]:
#surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [59]:
#surprise_results

In [60]:
#results_df = pd.DataFrame.from_dict(BaselineOnly.cv_results)

It looks like the baseline only model performed best and took the least amount of time. SVD was second best. Can we improve the SVD model with parameter tuning?

In [61]:
# params = {'n_factors': [10,20,50],
#          'reg_all': [ 0.02, 0.05, 0.07]}
# g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
# g_s_svd.fit(user_df)

In [62]:
# # print out optimal parameters for SVD after GridSearch
# print(g_s_svd.best_score)
# print(g_s_svd.best_params)

Best parameters for SVD were n_factors=10 and reg_all=0.05. The RMSE of this model is slightly better than for the BaselineOnly model, so it is currently our best model.

In [63]:
#train-test split the data
#do I do this 4 separate times?? (user, items, user features, and item features?)

trainset, testset = train_test_split(user_df, test_size=0.25)

In [64]:
user_item_ratings = user_data[['user_id', 'recipe_id', 'rating']]
user_item_ratings.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [65]:
#users_df_lists = user_diets[['user_id', 'recipe_id', 'rating']]
#users_df_lists.head()

In [66]:
users_df = user_data[['user_id', 'recipe_id', 'rating']]
users_df.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [67]:
items_df = recipe_data
items_df.head()

,name,recipe_id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,1,0
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,0,0
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,0,0
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,1,0
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,1,1


In [68]:
user_features_df = user_features.copy()

In [69]:
item_features_df = recipe_data[['recipe_id', 'vegetarian', 'vegan', 'GF']]

In [70]:
#for simplicity, renaming column 'recipe_id' to 'item_id'
items_df = items_df.copy().rename(columns={'recipe_id': 'item_id'})
item_features_df = item_features_df.copy().rename(columns={'recipe_id': 'item_id'})
user_item_ratings = user_item_ratings.copy().rename(columns={'recipe_id': 'item_id'})
users_df = user_item_ratings.copy().rename(columns={'recipe_id': 'item_id'})
items_df.head()

,name,item_id,minutes,tags,n_steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,1,0
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,0,0
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,0,0
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,1,0
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,1,1


In [71]:
#This is an alternative way to generate the necessary matrices - doing so by pivoting my df caused the new data to be too large for my computer

# Create a mapping between user/item IDs and matrix indices
user_id_to_index = {user_id: index for index, user_id in enumerate(users_df['user_id'])}
item_id_to_index = {item_id: index for index, item_id in enumerate(items_df['item_id'])}

In [72]:

# Initialize the ratings matrix
ratings_matrix = np.zeros((len(users_df), len(items_df)))
#ratings_matrix = np.zeros((len(user_features), len(items_df)))

# Fill in the ratings matrix based on user_item_ratings DataFrame
for _, row in users_df.iterrows():
  user_index = user_id_to_index.get(row['user_id'])
  item_index = item_id_to_index.get(row['item_id'])
  if user_index is not None and item_index is not None:
    ratings_matrix[user_index, item_index] = row['rating']



In [ ]:
#initial SVD without features: (computer crashes, doesnt work)
#from numpy import array
#from scipy.linalg import svd
## define a matrix
#A = ratings_matrix

# SVD
#U, s, VT = svd(ratings_matrix)
#print(U)
#print(s)
#print(VT)

In [74]:
#attempted to combine matrices for using additional user/item features
#print('Rounded Approximation of Ratings Matrix')
#np.round(ratings_matrix.dot(np.user_features_matrix(s).item_features_matrix(vt)))

Rounded Approximation of Ratings Matrix


AttributeError: module 'numpy' has no attribute 'user_features_matrix'

In [157]:
#matrix_A = ratings_matrix.dot(user_features_matrix).dot(item_features_matrix)
#A.dot(B).dot(C)

ValueError: shapes (1132367,231637) and (1132367,3) not aligned: 231637 (dim 1) != 1132367 (dim 0)

In [151]:
#define variables for class/function later on
num_users = len(set(user_item_ratings))
num_items = len(recipe_data)
num_user_features = len(user_features_df.columns) - 1  # Exclude user_id
num_item_features = len(item_features_df.columns) - 1  # Exclude item_id


In [ ]:
user_features_matrix = np.zeros((len(user_features_df), num_user_features))
item_features_matrix = np.zeros((len(item_features_df), num_item_features))
#user_index = users_df.get(row['user_id'])
#item_index = items_df.get(row['item_id'])
for _, row in user_features.iterrows():
   user_index = user_features.get(row['user_id'])
   if user_index is not None:
    user_features_matrix[user_index, :] = row.values[1:] # Assuming user features start from the second column

for _, row in item_features_df.iterrows():
    item_index = items_df.get(row['item_id'])
    if item_index is not None:
        item_features_matrix[item_index, :] = row.values[1:]  # Assuming item features start from the second column


In [155]:
user_features_matrix = user_features.iloc[:, 1:].values  # Assuming user features start from the second column
item_features_matrix = item_features_df.iloc[:, 1:].values  # Assuming item features start from the second column




In [ ]:
class SVDModel:
    def __init__(self, num_users, num_items, num_user_features, num_item_features, n_factors=10, reg_all=0.07, alpha=0.01, beta=0.01, iterations=1000):
        self.num_users = num_users
        self.num_items = num_items
        self.num_user_features = num_user_features
        self.num_item_features = num_item_features
        self.n_factors = n_factors
        self.reg_all = reg_all
        self.alpha = alpha  # Regularization term for user and item matrices
        self.beta = beta    # Learning rate
        self.iterations = iterations

    def fit(self, ratings, user_features, item_features):
        # Initialize user and item matrices
        #self.user_matrix = np.random.rand(self.num_users, self.n_factors)
        #self.item_matrix = np.random.rand(self.num_items, self.n_factors)

        # Initialize user and item feature matrices
        self.user_feature_matrix = np.random.rand(self.num_users, self.num_user_features)
        self.item_feature_matrix = np.random.rand(self.num_items, self.num_item_features)

        # Gradient descent for SVD model with features
        for _ in range(self.iterations):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if ratings[i][j] > 0:
                        error_ij = ratings[i][j] - np.dot(self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :]), self.item_matrix[j, :])

                        # Update user and item matrices
                        #self.user_matrix[i, :] += self.beta * (2 * error_ij * self.item_matrix[j, :] - self.alpha * self.user_matrix[i, :])
                        #self.item_matrix[j, :] += self.beta * (2 * error_ij * (self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) - self.alpha * self.item_matrix[j, :])

                        # Update user and item feature matrices
                        self.user_feature_matrix[i, :] += self.beta * (2 * error_ij * self.item_matrix[j, :] * self.item_feature_matrix[j, :] - self.alpha * self.user_feature_matrix[i, :])
                        self.item_feature_matrix[j, :] += self.beta * (2 * error_ij * (self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) * self.user_feature_matrix[i, :] - self.alpha * self.item_feature_matrix[j, :])

        # Predict ratings for all user-item pairs
        predicted_ratings = np.dot(self.user_matrix + np.dot(self.user_feature_matrix, self.item_feature_matrix.T), self.item_matrix.T)

        return predicted_ratings


In [ ]:
#SVD model with additional user and item features

class SVDModel:
    def __init__(self, num_users, num_items, num_user_features, num_item_features, latent_dim=10, alpha=0.01, beta=0.01, iterations=1000):
        self.num_users = num_users
        self.num_items = num_items
        self.num_user_features = num_user_features
        self.num_item_features = num_item_features
        self.latent_dim = latent_dim
        self.n_factors = n_factors
        self.reg_all = reg_all
        self.alpha = alpha  # Regularization term for user and item matrices
        self.beta = beta    # Learning rate
        self.iterations = iterations

    def fit(self, ratings, user_features, item_features):
        # Initialize user and item matrices
        self.user_matrix = np.random.rand(self.num_users, self.latent_dim)
        self.item_matrix = np.random.rand(self.num_items, self.latent_dim)

        # Initialize user and item feature matrices
        self.user_feature_matrix = np.random.rand(self.num_users, self.num_user_features)
        self.item_feature_matrix = np.random.rand(self.num_items, self.num_item_features)

        # Gradient descent for SVD model with features
        for _ in range(self.iterations):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if ratings[i][j] > 0:
                        error_ij = ratings[i][j] - np.dot(self.user_matrix[i, :] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :]), self.item_matrix[j, :])
                        for k in range(self.latent_dim):
                            self.user_matrix[i][k] += self.beta * (2 * error_ij * self.item_matrix[j][k] - self.alpha * self.user_matrix[i][k])
                            self.item_matrix[j][k] += self.beta * (2 * error_ij * (self.user_matrix[i][k] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) - self.alpha * self.item_matrix[j][k])

                        for l in range(self.num_user_features):
                            self.user_feature_matrix[i][l] += self.beta * (2 * error_ij * self.item_matrix[j][k] * self.item_feature_matrix[j][l] - self.alpha * self.user_feature_matrix[i][l])

                        for m in range(self.num_item_features):
                            self.item_feature_matrix[j][m] += self.beta * (2 * error_ij * (self.user_matrix[i][k] + np.dot(self.user_feature_matrix[i, :], self.item_feature_matrix[j, :])) * self.user_feature_matrix[i][k] - self.alpha * self.item_feature_matrix[j][m])

        # Predict ratings for all user-item pairs
        predicted_ratings = np.dot(self.user_matrix + np.dot(self.user_feature_matrix, self.item_feature_matrix.T), self.item_matrix.T)

        return predicted_ratings


In [ ]:
def run_svd(self):
        u, s, vt = svds(self.training_csc, k=self.k)

        self.Q = u
        diag_matrix = np.diag(s)
        self.P = diag_matrix.dot(vt)

In [ ]:
print(ratings_matrix.shape)
print(user_features_matrix.shape)
print(item_features_matrix.shape)

(1132367, 231637)
(1132367, 3)
(231637, 3)


In [ ]:
print("Shape of user_matrix + np.dot(user_feature_matrix, item_feature_matrix.T):", user_matrix.shape + np.dot(user_feature_matrix, item_feature_matrix.T).shape)
print("Shape of item_matrix.T:", item_matrix.T.shape)

NameError: name 'user_matrix' is not defined

In [ ]:
model = SVDModel(num_users, num_items, num_user_features, num_item_features, n_factors=10, reg_all=0.07)
predicted_ratings = model.fit(ratings_matrix, user_features_matrix, item_features_matrix)

print("Predicted Ratings:")
print(predicted_ratings)


ValueError: operands could not be broadcast together with shapes (3,10) (3,231637) 

In [ ]:
model = SVDModel(num_users, num_items, num_user_features, num_item_features)
predicted_ratings = model.fit(ratings_matrix, user_features_matrix, item_features_matrix)

# Make predictions (for example, predict ratings for all user-item pairs)
print("Predicted Ratings:")
print(predicted_ratings)


ValueError: operands could not be broadcast together with shapes (3,10) (3,231637) 

_______

In [ ]:
# Get the average rating for each user
#avg_ratings = user_ratings_table.mean(axis=1)

# Center each users ratings around 0
#user_ratings_table_centered = user_ratings_table.sub(avg_ratings, axis=0)

# Fill in the missing data with 0s
#user_ratings_table_normed = user_ratings_table_centered.fillna(0)

In [ ]:
#attempt this if all else fails?

#for col in user_item_ratings:
#    mean = user_item_ratings['rating'].mean()
#    user_item_ratings[col] = user_item_ratings[col].fillna(value=mean)
#user_item_ratings.head()

<ipython-input-64-4b0441f15a41>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_ratings[col] = user_item_ratings[col].fillna(value=mean)


,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['user_id'].plot(kind='hist', bins=20, title='user_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['recipe_id'].plot(kind='hist', bins=20, title='recipe_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_11['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['user_id'].plot(kind='line', figsize=(8, 4), title='user_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['recipe_id'].plot(kind='line', figsize=(8, 4), title='recipe_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['rating'].plot(kind='line', figsize=(8, 4), title='rating')
plt.gca().spines[['top', 'right']].set_visible(False)

ERROR:root:Did not find quickchart key chart-3cd6e4f6-4651-4d0c-9ca3-03299956adee in chart cache


In [ ]:
#ratings_matrix = user_item_ratings.pivot(index='user_id', columns='recipe_id', values='rating').fillna(0).values

In [ ]:
# Example usage
num_users = 5
num_items = 5
num_user_features = 3
num_item_features = 2

ratings = np.array([
    [5, 0, 3, 0, 2],
    [0, 4, 0, 0, 5],
    [1, 0, 0, 0, 4],
    [0, 2, 0, 4, 0],
    [3, 0, 0, 0, 0],
])

user_features = np.random.rand(num_users, num_user_features)
item_features = np.random.rand(num_items, num_item_features)

model = SVDModel(num_users, num_items, num_user_features, num_item_features)
predicted_ratings = model.fit(ratings, user_features, item_features)

print("Predicted Ratings:")
print(predicted_ratings)


In [ ]:
A = ratings_matrix
# Apply SVD
u, s, vt = svds(A, k=2) # k is the number of stretching factors

print ('A:\n', A.toarray())
print ('=')
print ('\nU:\n', u)
print ('\nΣ:\n', s)
print ('\nV.T:\n', vt)